In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [1]:
from datasets import load_dataset

In [3]:
s = load_dataset('Saptarshi7/PolicyQA')

Using custom data configuration plain_text-51df8be819373eff


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

ValueError: Couldn't cast
version: string
data: list<item: struct<title: string, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>>>
  child 0, item: struct<title: string, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>>
      child 0, title: string
      child 1, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>
          child 0, item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>